# Emma Programming Guide

## Preamble

Make sure you have build the *emma-tutorial* project with

```
mvn compile
```

before running the commands below.

### REPL Setup

Uncomment the following code if you want to include a manually deployed version of Emma.

In [ ]:
// interp.resolvers() = interp.resolvers() :+ ammonite.runtime.tools.Resolver.File(
//   "m2",
//   "/.m2/repository",
//   "/[organisation]/[module]/[revision]/[artifact]-[revision].[ext]",
//   m2 = true
// )

In [1]:
import ammonite.ops._
import java.nio.file.Paths

import $ivy.`org.emmalanguage:emma-language:0.2-rc1`
// `provided` dependencies expected by `emma-language`
import $ivy.`com.chuusai::shapeless:2.3.2`
import $ivy.`org.apache.hadoop:hadoop-common:2.8.0`
import $ivy.`org.apache.hadoop:hadoop-hdfs:2.8.0`
// compiler plugin required for `@emma.lib` annotation
import $plugin.$ivy.`org.scalamacros:paradise_2.11.8:2.0.1`

// project classpath
interp.load.cp(pwd / up / "emma-tutorial-library" / "target" / "classes")

import ammonite.ops._

import java.nio.file.Paths


import $ivy.$                                       
// `provided` dependencies expected by `emma-language`

import $ivy.$                             

import $ivy.$                                      

import $ivy.$                                    
// compiler plugin required for `@emma.lib` annotation

import $plugin.$                                           

// project classpath


### Example Data Setup

The following will download the [https://openflights.org]() data and import the data model (`data.openflights`) and some auxiliary functions (`lib.openflights`) used in the following examples. 

In [2]:
// project imports
import org.example.emma.tutorial.data.openflights._
import org.example.emma.tutorial.lib.openflights._

// download https://openflights.org data
downloadOpenFlightsData()

import org.example.emma.tutorial.data.openflights._

import org.example.emma.tutorial.lib.openflights._

// download https://openflights.org data


## Basic API

Emma programs require the following import.

In [3]:
import org.emmalanguage.api._

import org.emmalanguage.api._

### DataBag Basics

Parallel computation in Emma is represented by expressions over a generic type `DataBag[A]`. 
The type represents a distributed collection of elements of type `A` that do not have particular order and may contain duplicates.

`DataBag` instances are created directly from a Scala `Seq`

In [4]:
val squaresSeq = 1 to 42 map { x => (x, x * x) }
val squaresBag = DataBag(squaresSeq)

squaresSeq: collection.immutable.IndexedSeq[(Int, Int)] = Vector(
  (1, 1),
  (2, 4),
  (3, 9),
  (4, 16),
  (5, 25),
  (6, 36),
  (7, 49),
  (8, 64),
  (9, 81),
  (10, 100),
  (11, 121),
...
squaresBag: DataBag[(Int, Int)] = org.emmalanguage.api.ScalaSeq@8af4281

or by reading from a supported source, e.g. `CSV` or `Parquet`.

In [5]:
val csv = CSV(delimiter = ',')
val airports = DataBag.readCSV[Airport](file("airports.dat").toString, csv)
val airlines = DataBag.readCSV[Airline](file("airlines.dat").toString, csv)
val routes = DataBag.readCSV[Route](file("routes.dat").toString, csv)

csv: CSV = CSV(false, ',', ';', "UTF-8", Some('\"'), Some('\\'), None, "", 0)
airports: DataBag[Airport] = org.emmalanguage.api.ScalaSeq@6e1821c7
airlines: DataBag[Airline] = org.emmalanguage.api.ScalaSeq@f569b94e
routes: DataBag[Route] = org.emmalanguage.api.ScalaSeq@be6886da

Conversely, a `DataBag` can be converted back to a `Seq`

In [8]:
val squaresSeq = squaresBag.collect()

squaresSeq: Seq[(Int, Int)] = Vector(
  (1, 1),
  (2, 4),
  (3, 9),
  (4, 16),
  (5, 25),
  (6, 36),
  (7, 49),
  (8, 64),
  (9, 81),
  (10, 100),
  (11, 121),
...

or written to a supported file system.

In [7]:
airports.writeCSV(file("airports.copy.dat").toString, csv)
airlines.writeCSV(file("airlines.copy.dat").toString, csv)
routes.writeCSV(file("routes.copy.dat").toString, csv)

### Declarative Dataflows

In contrast to other distributed collection types such as Spark's `RDD` and Flink's `DataSet`, Emma's `DataBag` type is a proper monad. 
This means that joins and cross products in Emma can be declared using `for`-comprehension syntax in a manner akin to *Select-From-Where* expressions known from SQL.

In [ ]:
val flightsFromBerlin = for {
  al <- airlines
  if al.name == "Air Berlin"
  ap <- airports 
  if ap.city == Some("Berlin")
  rt <- routes
  if rt.airlineID == Some(al.id)
  if rt.srcID == Some(ap.id)
} yield (rt.src, rt.dst, rt.isShared)

flightsFromBerlin.collect().take(10)

In addition to comprehension syntax, the `DataBag` interface offers some combinators.
You can combine two `DataBag[A]` instances by taking their (duplicate preserving) `union` (corresponds to `UNION ALL` clause in SQL).

In [ ]:
val srcs = routes.map(_.src) 
val dsts = routes.map(_.dst)
val locs = srcs union dsts

You can eliminate duplicates with `distinct` (corresponds to the `DISTINCT` clause in SQL).

In [ ]:
val dupls = locs.collect().size
val dists = locs.distinct.collect().size

You can sample `N` elemens using [reservoir sampling](https://en.wikipedia.org/wiki/Reservoir_sampling).

In [ ]:
val sample = routes.map(_.src).sample(5)

You can extend all elements in a `DataBag` with a unique index.

In [ ]:
val indexed = routes.map(_.src).zipWithIndex.sample(5)

### Folds

The core parallel processing abstraction provided by `DataBag[A]` is a generic pattern for parallel
collection processing called *structural recursion*.   

To understand how structural recursion works, assume for a moment that `DataBag[A]` instances can be (conceptually) constructed in one of three ways: the `Empty` bag, a singleton bag `Singleton(x)`, or the union of two existing bags `Union(xs, ys)`. *Structural recursion* works on bags by

1. systematically deconstructing the input `DataBag[A]` instance, 
2. replacing the constructors with corresponding user-defined functions, and 
3. evaluating the resulting expression.

Formally, the above procedure can be specified second-order function called `fold` as follows.

```scala
def fold[B](zero: B)(init: A => B, plus: (B, B) => B): B = this match {
  case Empty         => zero
  case Singleton(x)  => init(x)
  case Union(xs, ys) => plus(xs.fold(e)(s, u), ys.fold(e)(s, u))
}
```

Note how `Empty` constructors are substituted by `zero` applications, `Singleton(x)` constructors are substituted by `init(x)`, and `Union(xs, ys)` by `plus(xs, ys)`.

A particular combination of `zero`, `init`, and `plus` function therefore defines a specific function. For example,

In [ ]:
val dupls = locs.fold(0L)(_ => 1L, _ + _)

is another way to compute the number of elements of `dupls`. Note that this expression will be evaluated **in parallel**, while the version we used above

In [ ]:
val dupls = locs.collect().size

first converts the `DataBag` **dupls** into a `Seq[String]` and count the number of elements locally.

A convenient way to bundle a specific combination of functions passed to a `fold` is through a dedicated trait.

```scala
// defined in `org.emmalanguage.api.alg`
trait Alg[-A, B] extends Serializable {
  val zero: B
  val init: A => B
  val plus: (B, B) => B
}
```

and overload `fold` as follows:

```scala
def fold[B](zero: B)(init: A => B, plus: (B, B) => B): B = this match {
  case Empty         => zero
  case Singleton(x)  => init(x)
  case Union(xs, ys) => plus(xs.fold(e)(s, u), ys.fold(e)(s, u))
}
```

Now, we can name commonly used triples as specific `Alg` instances.

```scala
object Size extends Alg[Any, Long] {
  val zero: Long = 0
  val init: Any => Long = const(1)
  val plus: (Long, Long) => Long = _ + _
}
```

and define corresponding aliases for the corresponding `fold(alg)` calls.

```scala
def size: Long = this.fold(Size)
```

The following `DataBag` methods are actually defined as specific folds.

In [ ]:
// cardinality tests
locs.size      // counts the number of elements
locs.nonEmpty  // checks if empty
locs.isEmpty   // checks if not empty

In [ ]:
// based on an implicit `Ordering`
locs.min       // minimum
locs.max       // maximum
locs.top(3)    // top-K
locs.bottom(3) // bottom-K

In [ ]:
// arithmetic operations
routes.map(_.stops).sum
DataBag(1 to 5).product

In [ ]:
// predicate testing
routes.count(_.stops < 3)
routes.forall(_.stops < 3)
routes.exists(r => r.src == "FRA" && r.dst == "SFO")
routes.find(r => r.src == "FRA" && r.dst == "SFO")

In [ ]:
// reducers
DataBag(1 to 5).reduce(1)(_ * _)
DataBag(1 to 5).reduceOption(_ * _)
DataBag(Seq.empty[Int]).reduceOption(_ * _)

## Code Parallelisation

To parallelise Emma code, you need to do two things

1. Setup a parallel dataflow framework (Flink or Spark)
2. quote an Emma code fragment 

### Parallel Dataflow Framework Setup

Run only one of the two options below.

#### Option 1: Spark (quote with `emma.onSpark`)

In [9]:
// `emma.onSpark` macro
import $ivy.`org.emmalanguage:emma-spark:0.2-rc1`
// `provided` dependencies expected by `emma-spark`
import $ivy.`org.apache.spark::spark-sql:2.1.0`

import $ivy.$                                    
// `provided` dependencies expected by `emma-spark`

import $ivy.$                                  

In [10]:
// required spark imports
import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.SparkSession

In [11]:
// implicit Spark environment
implicit val sparkSession = SparkSession.builder()
    .appName("Emma Programming Guide")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", Paths.get(sys.props("java.io.tmpdir"), "spark-warehouse").toUri.toString)
    .getOrCreate()

sparkSession: SparkSession = org.apache.spark.sql.SparkSession@35f2ae39

#### Option 2: Flink (quote with `emma.onFlink`)

TODO

### Quotations

Enclose a Scala code snippet in an `emma.onSpark` or `emma.onFlink` quotes.
The enclosed code block is optimized holistically by the Emma compiler, and `DataBag` expressions are offloaded to the corresponding parallel exeuction engine.

In [12]:
// override the `airports`, `routes`, and `airlines` members
// with the `def`-based versions from `data.openflights` module
import org.example.emma.tutorial.data.openflights._

import org.example.emma.tutorial.data.openflights._

In [16]:
// evaluates as Spark RDD map and filter
val berlinAirports = emma.onSpark {
  for {
    a <- airports
    if a.latitude > 52.3
    if a.latitude < 52.6
    if a.longitude > 13.2
    if a.longitude < 13.7
  } yield Location(
    a.name,
    a.latitude,
    a.longitude)
}

berlinAirports.collect().foreach(println)

Location(Berlin-Schönefeld International Airport,52.380001068115,13.522500038147)
Location(Berlin-Tempelhof International Airport,52.472999572753906,13.403900146484375)
Location(Berlin-Tegel International Airport,52.5597000122,13.2876996994)


berlinAirports: DataBag[Location[String]] = org.emmalanguage.api.SparkRDD@9d2a8543

In [ ]:
// evaluates as Spark RDD cascasde of joins
val rs = emma.onSpark {
  for {
    ap <- airports
    rt <- routes
    al <- airlines
    if rt.srcID == Some(ap.id)
    if rt.airlineID == Some(al.id)
  } yield (al.name, ap.country)
}

rs.sample(10).foreach(println)

In [ ]:
// evaluates as Spark RDD reduceByKey
val aggs = emma.onSpark {
  for {
    Group(k, g) <- routes.groupBy(_.src)
  } yield {
    val x = g.count(_.airline == "AB")
    val y = g.count(_.airline == "LH")
    k -> (x, y)
  }
}

aggs.withFilter({ case (k, (x, y)) => x > 0 && y > 0 }).sample(10).foreach(println)

## Code Modularity

To build domain-specific libraries based on Emma, enclose your function definitions in a top-level object and annotate this object with the `@emma.lib` annotation.

In [ ]:
@emma.lib
object hubs {
  def apply(M: Int) = {
    val rs = for {
      Group(k, g) <- ({
        routes.map(_.src)
      } union {
        routes.map(_.dst)
      }).groupBy(x => x)
      if g.size < M
    } yield k

    rs.collect().toSet
  }
}

@emma.lib
object reachable {
  def apply(N: Int)(hubs: Set[String]) = {
     val hubroutes = routes
       .withFilter(r => hubs(r.src) && hubs(r.dst))

     var paths = hubroutes
       .map(r => Path(r.src, r.dst))
     for (_ <- 0 until N) {
       val delta = for {
         r <- hubroutes; p <- paths if r.dst == p.src
       } yield Path(r.src, p.dst)
       paths = (paths union delta).distinct
     }

     paths
  }
}

In [ ]:
// override the `hubs` and `reachable` definitions from above
// with the identical implementations in the `lib.openflights`
// module in order to fix a bug in the type handling
import org.example.emma.tutorial.lib.openflights._

In [ ]:
// evaluates as Spark RDD reduceByKey
val rs = emma.onSpark {
  reachable(2)(hubs(50))
}

rs.sample(10).foreach(println)